In [1]:
import pandas as pd
import os
import glob
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category = FutureWarning)
from tqdm import tqdm

def get_path(country, week = 'march'):
    base = f'../../data/03-experiment/{country}/'
    path_tw = base + f'treatment/followers/00-raw/tweets/{week}/'
    rand = f'../../data/02-randomize/{country}/04-stratification/integrate/followers_randomized.parquet'
    baseline = base + 'baseline/00-raw/followers/tweets/'
    agg = base + f'treatment/followers/01-preprocess/'
    agg_base = base + 'baseline/01-preprocess/followers/'
    return path_tw, base, rand, baseline, agg, agg_base

In [2]:
# Batch 1
country = 'KE'
path_tw, base, rand, baseline, agg, agg_base = get_path(country, 'march')

if country == 'KE':
    n = 84
else:
    n = 74
    
df_final = pd.DataFrame()
for i in tqdm(range(0, n)):
    df = pd.read_parquet(f'{agg_base}predicted/baseline_{i}.parquet.gzip')
    df_final = pd.concat([df_final, df])

df_final = df_final.reset_index(drop=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [03:53<00:00,  2.78s/it]


In [4]:
df_final

,id,handle,author_id,created_at,text,lang,referenced_tweets,public_metrics.like_count,public_metrics.quote_count,public_metrics.reply_count,total_shares,total_reactions,total_comments,has_url,has_image,has_text,verifiability,true
0,1605174887693197312,000_benji,819486565835665410,2022-12-20T12:14:50.000Z,RT @AlbaMokopane: Jacob Zuma overestimated his...,en,"[{'id': '1604850928041005057', 'type': 'retwee...",0,0,0,67,67,0,1,0,1,1.0,1.0
1,1604818379520483328,000_benji,819486565835665410,2022-12-19T12:38:12.000Z,RT @Amika94855371: PRESIDENT RAMAPHOSA PLEASE ...,en,"[{'id': '1604795510019293184', 'type': 'retwee...",0,0,0,587,587,0,1,0,1,1.0,0.0
2,1634574718564028419,01010111000101O,1372105100,2023-03-11T15:19:16.000Z,No yellow\nNkt checked https://t.co/g8dDpkIbF8,en,"[{'id': '1634571065652400130', 'type': 'quoted'}]",0,0,1,0,1,1,1,0,1,1.0,0.0
3,1632770704427851776,01010111000101O,1372105100,2023-03-06T15:50:46.000Z,One was taking banned substances https://t.co/...,en,"[{'id': '1632470955040690176', 'type': 'quoted'}]",0,0,0,0,0,0,1,0,1,1.0,0.0
4,1632001089754021890,01010111000101O,1372105100,2023-03-04T12:52:35.000Z,RT @kagisodoc: SA National Parks Vacancies:\n\...,en,"[{'id': '1631714972358123522', 'type': 'retwee...",0,0,0,11,11,0,1,1,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18876920,1602215711698468864,zzz_hamed,1355449938772176901,2022-12-12T08:16:08.000Z,RT @CryptoHhhzzz: BlueSparrow - \n\n🔹Native to...,en,"[{'id': '1602215426490073088', 'type': 'retwee...",0,0,0,1,1,0,0,0,1,0.0,NaN
18876921,1602215692065013761,zzz_hamed,1355449938772176901,2022-12-12T08:16:03.000Z,RT @CryptoHhhzzz: BlueSparrow - \n\n🔹Native to...,en,"[{'id': '1602215472753422336', 'type': 'retwee...",0,0,0,1,1,0,0,0,1,0.0,NaN
18876922,1602215574079213568,zzz_hamed,1355449938772176901,2022-12-12T08:15:35.000Z,RT @zzz_hamed: BlueSparrow - \n\n🔹Native token...,en,"[{'id': '1602213955048554498', 'type': 'retwee...",0,0,0,1,1,0,0,0,1,0.0,NaN
18876923,1601925684846379015,zzz_hamed,1355449938772176901,2022-12-11T13:03:40.000Z,RT @BlueSparrowETH: The WANTED Sparrows! These...,en,"[{'id': '1601880570916737024', 'type': 'retwee...",0,0,0,39,39,0,0,0,1,0.0,NaN


In [3]:
df_final['RT'] = np.where((df_final['text'].str.contains('RT @', case=True, regex=False)) & 
                 (df_final['total_comments'] == 0), 1, 0)

df_final['post'] = np.where((~df_final['text'].str.contains('RT @', 
                    case=True, regex=False)) | 
                    (df_final['total_comments'] > 0), 1, 0)

df_count = df_final[['handle', 'author_id']].groupby(['author_id']).count()
df_count.rename({'handle': 'n_posts'}, axis=1, inplace=True)

df_agg = df_final[['author_id', 'RT', 'post']].groupby(['author_id']).sum().reset_index()

df_count = df_count.merge(df_agg, on = 'author_id', how='left')

first_month = df_final[(df_final['created_at'] > '2023-02-10')]

df_count1 = first_month[['handle', 'author_id']].groupby(['author_id']).count()
df_count1.rename({'handle': 'n_posts'}, axis=1, inplace=True)

df_agg1 = first_month[['author_id', 'RT', 'post']].groupby(['author_id']).sum().reset_index()

df_count1 = df_count1.merge(df_agg1, on = 'author_id', how='left')

In [7]:
round(np.percentile(df_count1['n_posts'], 95), 1)

843.0

In [4]:
bots = df_count1[df_count1['n_posts'] > round(np.percentile(df_count1['n_posts'], 95), 1)].reset_index(drop = True)
bots['dummy_1'] = 1

In [5]:
base1 = pd.read_parquet(f'../../data/04-analysis/{country}/baseline_features.parquet')
base1 = base1[['follower_id']]
base1.rename({'follower_id': 'author_id'}, axis=1, inplace=True)
base1 = base1.merge(df_count1, on = 'author_id', how = 'left')
base1 = base1.merge(bots[['author_id', 'dummy_1']], on = 'author_id', how = 'left')

for x in ['n_posts', 'RT', 'post', 'dummy_1']:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1['dummy_95'] = np.where(base1['n_posts'] > round(np.percentile(base1['n_posts'], 95), 1), 1, 0)
base1

,author_id,n_posts,RT,post,dummy_1,dummy_95
0,1000259098716368898,4.0,3.0,1.0,0.0,0
1,1000280756856545282,14.0,12.0,2.0,0.0,0
2,1000488233824866304,0.0,0.0,0.0,0.0,0
3,1000507757915787264,4.0,2.0,2.0,0.0,0
4,1001043873483952129,33.0,27.0,6.0,0.0,0
...,...,...,...,...,...,...
102529,997713498,2.0,1.0,1.0,0.0,0
102530,998604774,54.0,14.0,40.0,0.0,0
102531,999183930,9.0,0.0,9.0,0.0,0
102532,99934705,59.0,20.0,39.0,0.0,0


In [6]:
base1['dummy_95'].mean()

0.05000292585873954

In [9]:
round(np.percentile(base1['n_posts'], 95), 1)

345.3

In [10]:
base1[['author_id', 'dummy_95']].to_parquet(f'../../data/04-analysis/{country}/bots_batch1.parquet', index = False)

In [11]:
base1.to_parquet(f'../../data/03-experiment/{country}/baseline/baseline_info.parquet', index = False)

In [50]:
bots_completo = df_count[df_count['n_posts'] > round(np.percentile(df_count['n_posts'], 95), 1)].reset_index(drop = True)
bots_completo

,author_id,n_posts,RT,post
0,1000746698803761152,2468,878,1590
1,1001005867221823488,5219,2633,2586
2,1001786800329850885,2560,2274,286
3,1002162824234323969,4025,3694,331
4,1003259138,3114,1944,1170
...,...,...,...,...
4118,996423161053876224,9565,7887,1678
4119,996656006515052544,4194,1383,2811
4120,997141199310999552,2250,1345,905
4121,997479116432707584,1493,913,580


In [52]:
comparison = bots_completo.merge(bots[['author_id', 'dummy_1']], 
                                 on = 'author_id', how = 'outer')

In [53]:
second_month = df_final[(df_final['created_at'] < '2023-02-11') & 
                        (df_final['created_at'] > '2023-01-10')]

df_count2 = second_month[['handle', 'author_id']].groupby(['author_id']).count()
df_count2.rename({'handle': 'n_posts'}, axis=1, inplace=True)

df_agg2 = second_month[['author_id', 'RT', 'post']].groupby(['author_id']).sum().reset_index()

df_count2 = df_count2.merge(df_agg2, on = 'author_id', how='left')
bots2 = df_count2[df_count2['n_posts'] > round(np.percentile(df_count2['n_posts'], 95), 1)].reset_index(drop = True)
bots2['dummy_2'] = 1

comparison = comparison.merge(bots2[['author_id', 'dummy_2']], 
                              on = 'author_id', how = 'outer')

In [54]:
third_month = df_final[(df_final['created_at'] < '2023-01-11') & 
                        (df_final['created_at'] > '2022-12-10')]

df_count3 = third_month[['handle', 'author_id']].groupby(['author_id']).count()
df_count3.rename({'handle': 'n_posts'}, axis=1, inplace=True)

df_agg3 = third_month[['author_id', 'RT', 'post']].groupby(['author_id']).sum().reset_index()

df_count3 = df_count3.merge(df_agg3, on = 'author_id', how='left')
bots3 = df_count3[df_count3['n_posts'] > round(np.percentile(df_count3['n_posts'], 95), 1)].reset_index(drop = True)
bots3['dummy_3'] = 1

comparison = comparison.merge(bots3[['author_id', 'dummy_3']], 
                              on = 'author_id', how = 'outer')

In [55]:
comparison

,author_id,n_posts,RT,post,dummy_1,dummy_2,dummy_3
0,1000746698803761152,2468.0,878.0,1590.0,NaN,1.0,1.0
1,1001005867221823488,5219.0,2633.0,2586.0,1.0,1.0,1.0
2,1001786800329850885,2560.0,2274.0,286.0,1.0,1.0,NaN
3,1002162824234323969,4025.0,3694.0,331.0,1.0,1.0,1.0
4,1003259138,3114.0,1944.0,1170.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...
5092,99260413,NaN,NaN,NaN,NaN,NaN,1.0
5093,993798697657032709,NaN,NaN,NaN,NaN,NaN,1.0
5094,995931726034690048,NaN,NaN,NaN,NaN,NaN,1.0
5095,996729983983849473,NaN,NaN,NaN,NaN,NaN,1.0


### BATCH 2

In [2]:
country = 'KE'
if country == 'KE':
    n_end = 9
    n_base = 14
else:
    n_end = 7
    
path_tw, base, rand, baseline, agg, agg_base = get_path(country, 'march')

df_final = pd.DataFrame()
for i in tqdm(range(0, n_base)):
    df = pd.read_parquet(f'{agg_base}predicted/baseline_batch2_0{i}.parquet.gzip')
    df_final = pd.concat([df_final, df])

df_final1 = pd.DataFrame()
for i in tqdm(range(0, n_base)):
    df1 = pd.read_parquet(f'{agg_base}predicted/baseline2_batch2_{i}.parquet.gzip')
    df_final1 = pd.concat([df_final1, df1])

df_final = pd.concat([df_final, df_final1]).reset_index(drop=True)

df_final['RT'] = np.where((df_final['text'].str.contains('RT @', case=True, regex=False)) & 
                 (df_final['total_comments'] == 0), 1, 0)

df_final['post'] = np.where((~df_final['text'].str.contains('RT @', 
                    case=True, regex=False)) | 
                    (df_final['total_comments'] > 0), 1, 0)

first_month = df_final[(df_final['created_at'] > '2023-03-31')]

df_count = first_month[['lang', 'author_id']].groupby(['author_id']).count()
df_count.rename({'lang': 'n_posts'}, axis=1, inplace=True)

df_agg = first_month[['author_id', 'RT', 'post']].groupby(['author_id']).sum().reset_index()

df_count = df_count.merge(df_agg, on = 'author_id', how='left')

bots_batch2 = df_count[df_count['n_posts'] > 
                       round(np.percentile(df_count['n_posts'], 
                                           95), 1)].reset_index(drop = True)
bots_batch2['dummy_1'] = 1

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:25<00:00,  1.84s/it]


In [11]:
base1 = pd.read_parquet(f'../../data/04-analysis/{country}/baseline_batch2.parquet')
base1 = base1[['follower_id']]
base1.rename({'follower_id': 'author_id'}, axis=1, inplace=True)
base1 = base1.merge(df_count, on = 'author_id', how = 'left')
base1 = base1.merge(bots_batch2[['author_id', 'dummy_1']], on = 'author_id', how = 'left')

for x in ['n_posts', 'RT', 'post', 'dummy_1']:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1

,author_id,n_posts,RT,post,dummy_1
0,1251939178264354820,2.0,0.0,2.0,0.0
1,1252137750398197762,2.0,2.0,0.0,0.0
2,1252197986551169025,0.0,0.0,0.0,0.0
3,1252998418026442757,5.0,2.0,3.0,0.0
4,1253082749679939587,320.0,86.0,234.0,0.0
...,...,...,...,...,...
40189,99741477,55.0,16.0,39.0,0.0
40190,998212432903462912,463.0,30.0,433.0,0.0
40191,998687786143186944,4.0,3.0,1.0,0.0
40192,998796884000075776,544.0,37.0,507.0,0.0


In [14]:
base1['dummy_95'] = np.where(base1['n_posts'] > round(np.percentile(base1['n_posts'], 95), 1), 1, 0)

0.04998258446534309

In [16]:
base1[['author_id', 'dummy_95']].to_parquet(f'../../data/04-analysis/{country}/bots_batch2.parquet')

In [17]:
base1.to_parquet(f'../../data/03-experiment/{country}/baseline/baseline_info_batch2.parquet')

In [18]:
country = 'SA'
path_tw, base, rand, baseline, agg, agg_base = get_path(country, 'march')

df_final = pd.DataFrame()
for i in tqdm(range(0, 10)):
    df = pd.read_parquet(f'{agg_base}predicted/baseline_batch2_{i}.parquet.gzip')
    df_final = pd.concat([df_final, df])

df_final = df_final.reset_index(drop=True)

df_final['RT'] = np.where((df_final['text'].str.contains('RT @', case=True, regex=False)) & 
                 (df_final['total_comments'] == 0), 1, 0)

df_final['post'] = np.where((~df_final['text'].str.contains('RT @', 
                    case=True, regex=False)) | 
                    (df_final['total_comments'] > 0), 1, 0)

first_month = df_final

df_count = first_month[['lang', 'author_id']].groupby(['author_id']).count()
df_count.rename({'lang': 'n_posts'}, axis=1, inplace=True)

df_agg = first_month[['author_id', 'RT', 'post']].groupby(['author_id']).sum().reset_index()

df_count = df_count.merge(df_agg, on = 'author_id', how='left')

bots_batch2 = df_count[df_count['n_posts'] > 
                       round(np.percentile(df_count['n_posts'], 
                                           95), 1)].reset_index(drop = True)
bots_batch2['dummy_1'] = 1

#bots_batch2[['author_id', 'dummy_1']].to_parquet(f'../../data/04-analysis/{country}/bots_batch2.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.15s/it]


In [19]:
base1 = pd.read_parquet(f'../../data/04-analysis/{country}/baseline_batch2.parquet')
base1 = base1[['follower_id']]
base1.rename({'follower_id': 'author_id'}, axis=1, inplace=True)
base1 = base1.merge(df_count, on = 'author_id', how = 'left')
base1 = base1.merge(bots_batch2[['author_id', 'dummy_1']], on = 'author_id', how = 'left')

for x in ['n_posts', 'RT', 'post', 'dummy_1']:
    base1[x] = np.where(base1[x].isnull(), 0, base1[x])
    
base1

,author_id,n_posts,RT,post,dummy_1
0,1000263301182476289,14.0,13.0,1.0,0.0
1,1001406436478513152,0.0,0.0,0.0,0.0
2,1001833216402251776,24.0,24.0,0.0,0.0
3,1003813557073530880,13.0,0.0,13.0,0.0
4,1003945105517109249,1.0,1.0,0.0,0.0
...,...,...,...,...,...
28636,2270946743,91.0,84.0,7.0,0.0
28637,251925293,72.0,15.0,57.0,0.0
28638,2678104934,70.0,5.0,65.0,0.0
28639,73672445,642.0,83.0,559.0,0.0


In [20]:
base1['dummy_95'] = np.where(base1['n_posts'] > round(np.percentile(base1['n_posts'], 95), 1), 1, 0)

In [21]:
base1[['author_id', 'dummy_95']].to_parquet(f'../../data/04-analysis/{country}/bots_batch2.parquet')

In [22]:
base1.to_parquet(f'../../data/03-experiment/{country}/baseline/baseline_info_batch2.parquet')